In [127]:
## Solve Every Sudoku Puzzle

## See http://norvig.com/sudoku.html

## Throughout this program we have:
##   r is a row,    e.g. 'A'
##   c is a column, e.g. '3'
##   s is a square, e.g. 'A3'
##   d is a digit,  e.g. '9'
##   u is a unit,   e.g. ['A1','B1','C1','D1','E1','F1','G1','H1','I1']
##   grid is a grid,e.g. 81 non-blank chars, e.g. starting with '.18...7...
##   values is a dict of possible values, e.g. {'A1':'12349', 'A2':'8', ...}

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u])
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

################ Unit Tests ################

def test():
    "A set of tests that must pass."
    assert len(squares) == 81
    assert len(unitlist) == 27
    assert all(len(units[s]) == 3 for s in squares)
    assert all(len(peers[s]) == 20 for s in squares)
    assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                           ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                           ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
    assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                               'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                               'A1', 'A3', 'B1', 'B3'])
    print('All tests pass.')

################ Parse a Grid ################

def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    if len(chars) != 81: print(grid, chars, len(chars))
    assert len(chars) == 81
    return dict(zip(squares, chars))

################ Constraint Propagation ################

def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplaces[0], d):
                return False
    return values

################ Display as 2-D grid ################

def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width) + ('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    print()

################ Search ################

def solve(grid): return search(parse_grid(grid))

def search(values):
    "Using depth-first search and propagation, try all possible values."
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in squares):
        return values ## Solved!
    ## Chose the unfilled square s with the fewest possibilities
    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    for d in values[s]:
        result = search(assign(values.copy(), s, d))
        if result: return result

################ System test ################

import time

def solve_all(grids, name=''):
    """Attempt to solve a sequence of grids. Report results."""
    times, results = zip(*[time_solve(grid) for grid in grids])
    N = len(results)
    if N > 0:
        print("Solved %d of %d %s puzzles (avg %.2f secs (%d Hz), max %.2f secs)." % (
            sum(results), N, name, sum(times)/N, N/sum(times), max(times)))
            
def time_solve(grid):
    start = time.clock()
    values = solve(grid)
    t = time.clock()-start
    display(values)
    return (t, solved(values))

def solved(values):
    "A puzzle is solved if each unit is a permutation of the digits 1 to 9."
    def unitsolved(unit): return set(values[s] for s in unit) == set(digits)
    return values is not False and all(unitsolved(unit) for unit in unitlist)


grid1  = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2  = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
hard1  = '.....6....59.....82....8....45........3........6..3.54...325..6..................'


In [134]:
solve_all(examples)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 

2 4 5 |9 8 1 |3 7 6 
1 6 9 |2 7 3 |5 8 4 
8 3 7 |5 6 4 |2 1 9 
------+------+------
9 7 6 |1 2 5 |4 3 8 
5 1 3 |4 9 8 |6 2 7 
4 8 2 |7 3 6 |9 5 1 
------+------+------
3 9 1 |6 5 7 |8 4 2 
7 2 8 |3 4 9 |1 6 5 
6 5 4 |8 1 2 |7 9 3 

4 6 2 |8 3 1 |9 5 7 
7 9 5 |4 2 6 |1 8 3 
3 8 1 |7 9 5 |4 2 6 
------+------+------
1 7 3 |9 8 4 |2 6 5 
6 5 9 |3 1 2 |7 4 8 
2 4 8 |5 6 7 |3 1 9 
------+------+------
9 2 6 |1 7 8 |5 3 4 
8 3 4 |2 5 9 |6 7 1 
5 1 7 |6 4 3 |8 9 2 

1 3 7 |2 5 6 |8 4 9 
9 2 8 |3 1 4 |5 6 7 
4 6 5 |8 9 7 |3 1 2 
------+------+------
6 7 3 |5 4 2 |9 8 1 
8 1 9 |6 7 3 |2 5 4 
5 4 2 |1 8 9 |7 3 6 
------+------+------
2 5 6 |7 3 1 |4 9 8 
3 9 1 |4 2 8 |6 7 5 
7 8 4 |9 6 5 |1 2 3 

5 2 3 |8 1 6 |7 4 9 
7 8 4 |5 9 3 |1 2 6 
6 9 1 |4 7 2 |8 3 5 
------+--

In [136]:
from random import randint, sample, shuffle
from math import sqrt
import time
from __future__ import print_function
import copy
from numpy.random import choice

class SudokuSolver(object):
    def __init__(self, sudoku_size, init_values):
        self.sudoku_size = sudoku_size
        self.init_values = list(init_values)[:]
        self.fixed_positions = []
        self.population = []

        for i in range(0, len(self.init_values)):
            if not(self.init_values[i] == str(0) or self.init_values[i] == '.'):
                self.init_values[i] = int(self.init_values[i])
                self.fixed_positions.append(i)
            else:
                self.init_values[i] = 0

    def nodes(self):
        return self.sudoku_size * self.sudoku_size

    def kcolors(self):
        return self.sudoku_size

    def edges(self):
        return (self.nodes() * (self.kcolors()-1 + (self.kcolors()-(sqrt(self.kcolors()))) * 2)) / 2

    def population(self):
        return self.population

    def printGrid(self, solution, printfile=None):
        for i in range(0, 9):
            if printfile is not None:
                print(solution[i*9:i*9+9], file=printfile)
            else:
                print(solution[i*9:i*9+9])

    def row_indexes(self, position):
        return range(int(position/self.sudoku_size)*9,int(position/self.sudoku_size)*9+9)

    def row_index(self, position):
        return int(position/self.sudoku_size)

    def col_index(self, position):
        return position%self.sudoku_size

    def col_indexes(self, position):
        return range(position%self.sudoku_size,73+(position%self.sudoku_size),9)

    def zone_number(self, position):
        row_index = self.row_index(position)
        col_index = self.col_index(position)
        if (row_index % 9 < 3):
            if (col_index % 9 < 3):
                return 0
            elif (col_index % 9 < 6):
                return 1
            else:
                return 2
        elif (row_index % 9 < 6):
            if (col_index % 9 < 3):
                return 3
            elif (col_index % 9 < 6):
                return 4
            else:
                return 5
        else:
            if (col_index % 9 < 3):
                return 6
            elif (col_index % 9 < 6):
                return 7
            else:
                return 8

    def zone_indexes(self, zone_number):
        first = self.sudoku_size * int(zone_number/3)*3 + zone_number % 3 * 3
        indexes = []
        for i in range(0, 3):
            indexes.append(first) 
            indexes.append(1 + first)
            indexes.append(2 + first)
            first = first + self.sudoku_size
        return indexes

    def zone_indexes_by_pos(self, position):
        zone_number = self.zone_number(position)
        return self.zone_indexes(zone_number)

    def adyacent_indexes(self, position):
        indexes = set()
        [indexes.add(x) for x in self.col_indexes(position)]
        [indexes.add(x) for x in self.row_indexes(position)]
        [indexes.add(x) for x in self.zone_indexes_by_pos(position)]
        indexes.remove(position)
        return indexes

    def fitness(self, solution):
        # Cantidad de ejes que unen nodos del mismo color
        bad_edges = 0
        for pos in range(0, self.nodes()):
            for ady in self.adyacent_indexes(pos):
                if (solution[ady] == solution[pos]):
                    bad_edges += 1
        return (bad_edges / 2)

    def initPopulation(self, n_population):
        self.population = []
        for i in range(0, n_population):
            solution = self.createRandomSol()
            self.population.append(copy.copy(solution))

    def createRandomSol(self):
        solution = self.init_values
        indexes = list(range(0,81))
        shuffle(indexes)
        while (len(indexes) > 0):
            i = indexes.pop()
            if not(i in self.fixed_positions):
                solution[i] = self.getAvailableColor(solution, i, allow_random=True)
        return solution
    
    def getAvailableColor(self, solution, pos, allow_random=True):
        available_colors = set(range(1,10))
        for i in self.adyacent_indexes(pos):
            if solution[i] in available_colors:
                available_colors.remove(solution[i])
        if len(available_colors) > 0:
            return sample(available_colors, 1)[0]
        elif allow_random:
            return randint(1,9)
        else:
            return solution[pos]

    def select_parents(self):
        n_population = len(self.population)
        parent1_index = randint(0, int(n_population/2))
        parent2_index = randint(0, int(n_population/2))
        
        while (parent1_index == parent2_index):
            parent2_index = randint(0, int(n_population/2))

        return self.population[parent1_index], self.population[parent2_index]

    def crossover1(self, parent1, parent2):
    # Por zonas
        solution = [0] * 81
        for zone_number in range(0,9):
            r = randint(0,1)
            if (r):
                parent = parent1
            else:
                parent = parent2
            indexes = self.zone_indexes(zone_number)
            for i in indexes:
                solution[i] = parent[i]
        return solution

    def crossover2(self, parent1, parent2):
        # Por filas
        solution = [0] * 81
        for row in range(0,9):
            r = randint(0,1)
            if (r):
                parent = parent1
            else:
                parent = parent2
            for i in range(0,9):
                solution[row*9 + i] = parent[row*9 + i]
        return solution

    def crossover3(self, parent1, parent2):
        # Por columnas
        solution = [0] * 81
        for col in range(0,9):
            r = randint(0,1)
            if (r):
                parent = parent1
            else:
                parent = parent2
            for i in range(0,9):
                solution[col + i*9] = parent[col + i*9]
        return solution

    def mutation(self, solution, allow_random_coloring=False):
        # Recoloreo nodos en orden random, con la opcion de colorear con colores random
        indexes = list(range(0,81))
        shuffle(indexes)
        while (len(indexes) > 0):
            i = indexes.pop()
            if not(i in self.fixed_positions):
                solution[i] = self.getAvailableColor(solution, i, allow_random=allow_random_coloring)
        return solution

    def mutation_bad_colored(self, solution, allow_random_coloring=False):
        bad_colored = []
        for i in range(0, len(solution)):
            for ady in self.adyacent_indexes(i):
                if solution[i] == solution[ady]:
                    bad_colored.append(i)
        shuffle(bad_colored)
        for i in bad_colored:
            solution[i] = self.getAvailableColor(solution, i, allow_random=allow_random_coloring)
        return solution

    def solve(self, n_population):
        
        # Inicializacion de variables
        start_time = time.time()
        self.initPopulation(n_population)
        elite = [] # usado para los restarts
        best_fitness = self.edges()
        iter_count = 0
        iter_without_improvement = 0

        while (best_fitness > 0 and iter_count < 1000 and iter_without_improvement < 10000):

            # Actualizacion del best fitness
            self.population = sorted(self.population, key=self.fitness)
            new_best = self.fitness(self.population[0])
            if (new_best < best_fitness):
                best_fitness = new_best
                iter_without_improvement = 0
                # print("iteration %d, new best_fitness %d" %(iter_count, best_fitness))
                # self.printGrid(self.population[0])
                if best_fitness == 0: 
                    break
            else:
                iter_without_improvement += 1

            # Actualizacion de la poblacion
            for i in range(int(n_population/2), n_population):

                # Generacion del hijo, evito que se generen repetidos
                son_iguales_mutaciones = True
                intentos = 0
                while (son_iguales_mutaciones):
                    
                    # Seleccion de padres
                    parent1, parent2 = self.select_parents()

                    # Crossover
                    opt_crossover = randint(1,3)
                    if opt_crossover == 1:
                        child = self.crossover1(parent1, parent2)
                    elif opt_crossover == 2:
                        child = self.crossover2(parent1, parent2)
                    else:
                        child = self.crossover3(parent1, parent2)

                    # Mutacion
                    allow = iter_without_improvement > 5
                    child = self.mutation(child, allow_random_coloring = allow) 

                    # Chequeo si es repetido
                    son_iguales_mutaciones = False
                    for p in self.population :  
                        if p == child : 
                            son_iguales_mutaciones = True
                            intentos += 1
                            break

                    # Si pasa el nro max de intentos repetidos genero un hijo random        
                    if intentos > 5 :
                        child = self.createRandomSol()
                        son_iguales_mutaciones = False
                    
                    if not(son_iguales_mutaciones):
                        self.population[i] = copy.copy(child)
                        
            # Restarts, para destrabar y resetear el algoritmo
            if iter_without_improvement > 30:
                for i in range(0, 10):
                    elite.append(self.mutation_bad_colored(copy.copy(self.population[i]), allow_random_coloring=True))
                self.initPopulation(n_population)
                for i in range(0, len(elite)):
                    self.population[i] = elite[i]
                best_fitness = self.edges()
                iter_without_improvement = 0

            # Condicion de parada
            if iter_without_improvement == 200 or time.time() - start_time > 240:
                break

            iter_count = iter_count + 1

        # Resultado
        if (best_fitness == 0):
            print("--- Se resolvió el SUDOKU en %s segundos!!!! ---" % (time.time() - start_time))
            self.printGrid(self.population[0])
            return 1
        else:
            print("NO ANDUVO, se obtuvo un fitness de %d luego de %s segundos :(" % (best_fitness, (time.time() - start_time)))
            return 0

In [123]:
init_values_easy = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
init_values_hard = '.....6....59.....82....8....45........3........6..3.54...325..6..................'
grid1  = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'

s = SudokuSolver(9, init_values_easy)
s.solve(200)

iteration 0, new best_fitness 29
[4, 5, 7, 1, 6, 1, 8, 9, 5]
[6, 3, 1, 5, 9, 6, 4, 2, 4]
[2, 4, 9, 7, 3, 4, 6, 7, 1]
[4, 2, 8, 5, 8, 1, 3, 6, 7]
[1, 9, 6, 3, 8, 7, 4, 5, 9]
[3, 7, 5, 9, 1, 6, 2, 8, 4]
[8, 9, 2, 6, 4, 3, 1, 7, 1]
[5, 6, 3, 2, 2, 5, 9, 4, 8]
[1, 1, 4, 4, 7, 8, 5, 3, 2]
iteration 1, new best_fitness 25
[4, 7, 2, 1, 6, 8, 8, 3, 5]
[5, 3, 9, 8, 2, 4, 6, 3, 7]
[8, 2, 1, 7, 3, 6, 2, 9, 4]
[4, 2, 3, 4, 9, 5, 7, 6, 8]
[7, 9, 5, 3, 8, 6, 4, 1, 2]
[6, 5, 8, 3, 1, 7, 9, 5, 9]
[9, 8, 3, 6, 5, 3, 1, 7, 1]
[5, 6, 7, 2, 4, 1, 5, 8, 6]
[1, 4, 4, 8, 7, 9, 3, 2, 9]
iteration 2, new best_fitness 21
[4, 9, 7, 3, 2, 1, 8, 6, 5]
[6, 3, 1, 8, 9, 5, 9, 2, 7]
[8, 5, 2, 7, 4, 6, 3, 1, 3]
[9, 2, 3, 5, 1, 4, 7, 6, 7]
[7, 1, 6, 9, 8, 2, 4, 5, 3]
[8, 4, 5, 6, 1, 7, 5, 9, 2]
[2, 8, 9, 6, 5, 3, 1, 7, 4]
[5, 6, 7, 2, 4, 3, 9, 8, 9]
[1, 6, 4, 4, 7, 9, 6, 3, 2]
iteration 3, new best_fitness 19
[4, 6, 2, 1, 3, 9, 8, 5, 5]
[3, 3, 9, 8, 6, 5, 4, 1, 7]
[1, 5, 8, 7, 2, 4, 3, 4, 6]
[7, 2, 7, 3, 3, 9, 5, 6, 8]


1

In [ ]:
from __future__ import print_function

def test_sudoku(grid, dificulty, iterations = None, max_time = None):
    
    if iterations is None:
        iterations = 1
    
    tiempos_resueltos = []

    for i in range(0, iterations):
        start_time = time.time()
        
        s = SudokuSolver(9, grid)
        res = s.solve(200)
        
        duracion = time.time() - start_time

        if (res):
            tiempos_resueltos.append(duracion)
        
        else:
            # imprimir poblacion final a un archivo para tratar de averiguar por que se traba
            filename = "population_dif" + str(dificulty) + "_it" + str(i) + ".txt"
            with open(filename, 'w') as f:
                for p in s.population:
                    s.printGrid(p, printfile=f)
                    print ("fitness %d" % s.fitness(p), file=f)
                    print ('-------------------------------------', file=f)

    return tiempos_resueltos

In [52]:
examples_txt = open("sudoku_examples.txt").readlines()

examples = []
for i in range(1,500,10):
    example = ''
    for j in range(0,9):
        example = example + examples_txt[i+j].strip()
    examples.append(example)

In [126]:
i = 0
# for s in list(reversed(examples)):
for s in examples:
    test_sudoku(s, i)
    i +=1

--- Se resolvió el SUDOKU en 1.72734904289 segundos!!!! ---
--- Se resolvió el SUDOKU en 12.5782001019 segundos!!!! ---
--- Se resolvió el SUDOKU en 12.4390389919 segundos!!!! ---
--- Se resolvió el SUDOKU en 12.0998849869 segundos!!!! ---
--- Se resolvió el SUDOKU en 0.769387960434 segundos!!!! ---
--- Se resolvió el SUDOKU en 18.8476428986 segundos!!!! ---
--- Se resolvió el SUDOKU en 15.1306979656 segundos!!!! ---
--- Se resolvió el SUDOKU en 0.889339923859 segundos!!!! ---
--- Se resolvió el SUDOKU en 19.2715761662 segundos!!!! ---
--- Se resolvió el SUDOKU en 15.8783631325 segundos!!!! ---
--- Se resolvió el SUDOKU en 2.48813700676 segundos!!!! ---
--- Se resolvió el SUDOKU en 2.75608706474 segundos!!!! ---
--- Se resolvió el SUDOKU en 11.7142958641 segundos!!!! ---
--- Se resolvió el SUDOKU en 15.2610228062 segundos!!!! ---
--- Se resolvió el SUDOKU en 51.0102241039 segundos!!!! ---
--- Se resolvió el SUDOKU en 0.753906965256 segundos!!!! ---
--- Se resolvió el SUDOKU en 0.280414

In [137]:
s = SudokuSolver(9, examples[-1])
s.solve(200)

--- Se resolvió el SUDOKU en 44.1842281818 segundos!!!! ---
[3, 4, 1, 2, 8, 5, 9, 6, 7]
[5, 9, 8, 1, 6, 7, 2, 4, 3]
[7, 2, 6, 9, 3, 4, 5, 1, 8]
[6, 7, 3, 4, 5, 9, 1, 8, 2]
[9, 8, 5, 6, 2, 1, 7, 3, 4]
[4, 1, 2, 8, 7, 3, 6, 5, 9]
[8, 6, 9, 5, 4, 2, 3, 7, 1]
[2, 3, 4, 7, 1, 6, 8, 9, 5]
[1, 5, 7, 3, 9, 8, 4, 2, 6]


1